In [1]:
sudo apt-get install openjdk-8-jre

SyntaxError: invalid syntax (<ipython-input-1-cc5eac366280>, line 1)

In [2]:

from konlpy.tag import Twitter
from gensim.models import Word2Vec
import csv //comma separated values, txt형 파일확장자
 
twitter = Twitter()
 
file = open("Article_shuffled.csv", 'r', encoding='euc-kr')
    //file에=열어서 저장해라("Article_shuffled를")
line = csv.reader(file)
token = []
embeddingmodel = []
 
for i in line:
    sentence = twitter.pos(i[0], norm=True, stem=True)
        //sentence에 = twitter.내장함수인pos(part of speech=형태소)를(line 별로 저장)
    temp = []
    temp_embedding = []
    all_temp = []
    for k in range(len(sentence)):
        temp_embedding.append(sentence[k][0])
            //temp_embedding에는 index 정보 저장
        temp.append(sentence[k][0] + '/' + sentence[k][1])
            //temp에는 index 정보/날짜정보 저장
        //for 문을 돌면 temp_embedding에는 리스트별 index 정보가, temp에는 리스트별 index/날짜 정보가
    all_temp.append(temp)
        //all_temp에는 temp저장
    embeddingmodel.append(temp_embedding)
        //embeddingmodel에는 temp_embedding 저장
    category_number_dic = {'IT과학': 0, '경제': 1, '정치': 2,'세계':3,'오피니언':4, '사회': 5, '생활문화': 6}
    all_temp.append(category_number_dic.get(category))
        //all_temp에 카테고리(it과학:0)정보까지 저장
    token.append(all_temp)
print("토큰 처리 완료")
    //여기까지 하면 all_temp에 index + 날짜 + 카테고리dic 정보 저장되어있음
 
embeddingmodel = []
for i in range(len(token)): ////index를 돌고
    temp_embeddingmodel = []
    for k in range(len(token[i][0])): ////날짜를 돌고
        temp_embeddingmodel.append(token[i][0][k])
    embeddingmodel.append(temp_embeddingmodel)
        ////돌면서 temp_embeddingmodel에 넣고
// max_vocab size 10000000 개당 1 GB 메모리 차지
embedding = Word2Vec(embeddingmodel, size=300, window=5, min_count=10, iter=5, sg=1, max_vocab_size = 360000000)
embedding.save('post.embedding') //만들어진 사각형 숫자 정보를 -> vector로 바꾸는 모델
 


IndentationError: unexpected indent (<ipython-input-2-892f36910589>, line 9)

In [3]:
# -*- coding: utf-8 -*-
"""
Created on Mon May 21 14:30:52 2018

@author: jbk48
"""

import tensorflow as tf

class Bi_LSTM():
    
    def __init__(self, lstm_units, num_class, keep_prob):
        
        self.lstm_units = lstm_units
        
        with tf.variable_scope('forward', reuse = tf.AUTO_REUSE):
            
            self.lstm_fw_cell = tf.nn.rnn_cell.LSTMCell(lstm_units, forget_bias=1.0, state_is_tuple=True)
            self.lstm_fw_cell = tf.contrib.rnn.DropoutWrapper(self.lstm_fw_cell, output_keep_prob = keep_prob)
            
        with tf.variable_scope('backward', reuse = tf.AUTO_REUSE):
            
            self.lstm_bw_cell = tf.nn.rnn_cell.LSTMCell(lstm_units, forget_bias=1.0, state_is_tuple=True)
            self.lstm_bw_cell = tf.contrib.rnn.DropoutWrapper(self.lstm_fw_cell, output_keep_prob = keep_prob)
        
        with tf.variable_scope('Weights', reuse = tf.AUTO_REUSE):
           
            self.W = tf.get_variable(name="W", shape=[2 * lstm_units, num_class],
                                dtype=tf.float32, initializer = tf.contrib.layers.xavier_initializer())
            self.b = tf.get_variable(name="b", shape=[num_class], dtype=tf.float32,
                                initializer=tf.zeros_initializer())
            
            
    def logits(self, X, W, b, seq_len):
        
        (output_fw, output_bw), states = tf.nn.bidirectional_dynamic_rnn(self.lstm_fw_cell, self.lstm_bw_cell,dtype=tf.float32,
                                                                            inputs = X, sequence_length = seq_len)
        ## concat fw, bw final states
        outputs = tf.concat([states[0][1], states[1][1]], axis=1)
        pred = tf.matmul(outputs, W) + b        
        return pred
        
    def model_build(self, logits, labels, learning_rate = 0.001):
        
        with tf.variable_scope("loss"):
            
            loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = logits , labels = labels)) # Softmax loss
            optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss) # Adam Optimizer
            
        return loss, optimizer
    
    def graph_build(self, avg_loss, avg_acc):
        
        tf.summary.scalar('Loss', avg_loss)
        tf.summary.scalar('Accuracy', avg_acc)
        merged = tf.summary.merge_all()
        return merged



In [4]:
"""
@author: MsWon
@editor: lumyjuwon
"""

import time
import tensorflow as tf
import numpy as np
import Bi_LSTM as Bi_LSTM
import Word2Vec as Word2Vec
import csv
from konlpy.tag import Twitter
import os

twitter = Twitter()
W2V = Word2Vec.Word2Vec()

file = open("Article_shuffled.csv", 'r', encoding='euc-kr')
line = csv.reader(file)
token = []
embeddingmodel = []

for i in line: ///line으로 자르기
    content = i[3]  # csv에서 뉴스 제목 또는 뉴스 본문 column으로 변경
    sentence = twitter.pos(i[0], norm=True, stem=True) ///line안에 형태소별로 자르기
    temp = []
    temp_embedding = []
    all_temp = []
    for k in range(len(sentence)):
        temp_embedding.append(sentence[k][0]) ///temp_embedding에는 (형태소들 저장)
        temp.append(sentence[k][0] + '/' + sentence[k][1]) /// temp에는 (형태소[0] / 해당품사[1] 저장)
    all_temp.append(temp) 
    embeddingmodel.append(temp_embedding)
    category = i[1]  # csv에서 category column으로 변경
    ///for문 다 돌면 all_temp에는 모든 문장의 형태소[0]/품사[1]가 저장 , embeddingmodel에는 모든 문장의 품사[1]가 저장
    category_number_dic = {'IT과학': 0, '경제': 1, '정치': 2,'세계':3,'오피니언':4, '사회': 5, '생활문화': 6}
    all_temp.append(category_number_dic.get(category)) ///all_temp에 카테고리 매핑정보까지 저장
    token.append(all_temp)
print("토큰 처리 완료")
    ////앞과정과 동일하게 토큰 만들기

tokens = np.array(token)
print("token 처리 완료")
print("train_data 최신 버전인지 확인")
train_X = tokens[:, 0] ///형태소들
train_Y = tokens[:, 1] ///품사들

train_Y_ = W2V.One_hot(train_Y)  ///품사들도 vec형태로
train_X_ = W2V.Convert2Vec("Data\\post.embedding",train_X) ///만들어둔embedingmodel 가져오기

Batch_size = 32
Total_size = len(train_X)
Vector_size = 300
seq_length = [len(x) for x in train_X]
Maxseq_length = max(seq_length)
learning_rate = 0.001
lstm_units = 128
num_class = 12
training_epochs = 5
keep_prob = 0.75
///머신러닝 기본 요소들 설정

X = tf.placeholder(tf.float32, shape = [None, Maxseq_length, Vector_size], name = 'X')
Y = tf.placeholder(tf.float32, shape = [None, num_class], name = 'Y')
seq_len = tf.placeholder(tf.int32, shape = [None])

BiLSTM = Bi_LSTM.Bi_LSTM(lstm_units, num_class, keep_prob)

with tf.variable_scope("loss", reuse = tf.AUTO_REUSE): //공유 변수(W와 B가 너무 많을 때 한번에 관리해줌)
    logits = BiLSTM.logits(X, BiLSTM.W, BiLSTM.b, seq_len) //logistic 회귀 모델을 이용한 BiLSTM
    loss, optimizer = BiLSTM.model_build(logits, Y, learning_rate) //학습 변수로(logit,Y,leanring rate넣기)

prediction = tf.nn.softmax(logits) //softmax : 결과값을 0과 1 사이로 만들어주는 함수
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1)) //correct_prediction : 예측값과 실제라벨링값이 일치하는가?
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32)) //일치하면1을 증가시켜서 accuracy 측정

init = tf.global_variables_initializer() //변수 초기화

total_batch = int(Total_size / Batch_size) //전체 batch_size 결정

print("Start training!")

modelName = "BiLSTM.model"
saver = tf.train.Saver()


with tf.Session() as sess:

    start_time = time.time()
    sess.run(init)
    train_writer = tf.summary.FileWriter('Bidirectional_LSTM', sess.graph)
    i = 0
    for epoch in range(training_epochs): ///epoch 횟수만큼

        avg_acc, avg_loss = 0. , 0.
        for step in range(total_batch):

            train_batch_X = train_X_[step*Batch_size : step*Batch_size+Batch_size] //+Batch_size만큼씩 늘려가면서 학습
            train_batch_Y = train_Y_[step*Batch_size : step*Batch_size+Batch_size] //+Batch_size만큼씩 늘려가면서 학습
            batch_seq_length = seq_length[step*Batch_size : step*Batch_size+Batch_size]
            
            train_batch_X = W2V.Zero_padding(train_batch_X, Batch_size, Maxseq_length, Vector_size) //zero_padding적용
            
            sess.run(optimizer, feed_dict={X: train_batch_X, Y: train_batch_Y, seq_len: batch_seq_length})
            //W 최적화 진행
            loss_ = sess.run(loss, feed_dict={X: train_batch_X, Y: train_batch_Y, seq_len: batch_seq_length})
            avg_loss += loss_ / total_batch
            //loss 함수값 계산
            
            acc = sess.run(accuracy , feed_dict={X: train_batch_X, Y: train_batch_Y, seq_len: batch_seq_length})
            avg_acc += acc / total_batch
            //accuracy 값 계산
            print("epoch : {:02d} step : {:04d} loss = {:.6f} accuracy= {:.6f}".format(epoch+1, step+1, loss_, acc))

        summary = sess.run(BiLSTM.graph_build(avg_loss, avg_acc))       
        train_writer.add_summary(summary, i)
        i += 1

        ///////////시간 정보 알려주는 추가 설명////////////
    duration = time.time() - start_time
    minute = int(duration / 60)
    second = int(duration) % 60
    print("%dminutes %dseconds" % (minute,second))
    save_path = saver.save(sess, os.getcwd())

    train_writer.close()
    print('save_path',save_path)



SyntaxError: invalid syntax (<ipython-input-4-ae642fb00228>, line 23)

In [6]:
import csv
import os

os.chdir("C:\\Users\\user\\Downloads")

category = ['IT과학','경제','정치','세계','오피니언','사회','생활문화']

file_unity = open('Article_unity.csv', 'w', encoding='euc-kr')
//열어서_저장해라(Article_unity라는 새로운 파일, 쓰기전용으로, 번역_인코딩은euc-kr유니코드한국어로)
wcsv = csv.writer(file_unity)
//WriteCSV에=써넣어라(file_unity를)
count = 0

for category_element in category: 
        //위에 IT과학 경제 정치 ... 를 한 요소씩 반복
    file = open('Article_'+category_element+'.csv','r',encoding='euc-kr',newline="")
        //file에 = 열어서_저장해라(형식이 'Article_정치.csv'인 애를, 읽기전용으로,번역_인코딩은euc-kr유니코드한국어로, newline에는 빈 벡터를 두고)
    line = csv.reader(file)
        //line에 = 읽어들여라(file을 즉:정치 기사리스트를 한 줄씩)
    try : 
        for line_text in line:
                //line을 한 줄씩 반복(리스트를 반복)
            wcsv.writerow([line_text[1], line_text[2], line_text[3], line_text[4]])
                //아까 비어있던 file_unity를 넣은 WCSV에.한줄씩써넣어라(엑셀의[카테고리], 엑셀의[뉴스], 엑셀[본문] )
    except:
        pass


IndentationError: unindent does not match any outer indentation level (<tokenize>, line 16)

In [7]:
import csv
import random
import os
 
os.chdir("C:\\Users\\Juwon\\PycharmProjects\\tensorflows\\parser\\Csv") // Csv가 있는 경로 설정
 
file = open('Article_unity.csv', 'r', encoding='euc-kr')
line = file.readlines()
random.shuffle(line)
rcsv = csv.reader(line)
 
file_write = open('Article_shuffled.csv', 'w', encoding='euc-kr', newline="")
wcsv = csv.writer(file_write)
 
for i in rcsv:
    try:
        wcsv.writerow([i[0].strip(), i[1], i[2], i[3]])
		////아까 비어있던 file_unity를 넣은 WCSV에.한줄씩써넣어라(엑셀의 [날짜].\n지우고, 엑셀의[카테고리], 엑셀의[뉴스], 엑셀[본문] )
    except:
        pass
 

SyntaxError: invalid syntax (<ipython-input-7-2e636a750cb6>, line 5)

In [ ]:
from konlpy.tag import Twitter
from gensim.models import Word2Vec
import csv //comma separated values, txt형 파일확장자
 
twitter = Twitter()
 
file = open("Article_shuffled.csv", 'r', encoding='euc-kr')
    //file에=열어서 저장해라("Article_shuffled를")
line = csv.reader(file)
token = []
embeddingmodel = []
 
for i in line:
    sentence = twitter.pos(i[0], norm=True, stem=True)
        //sentence에 = twitter.내장함수인pos(part of speech=형태소)를(line안에 형태소별로 자르기)
    temp = []
    temp_embedding = []
    all_temp = []
    for k in range(len(sentence)):
        temp_embedding.append(sentence[k][0])
            ///temp_embedding에는 (형태소들 저장)
        temp.append(sentence[k][0] + '/' + sentence[k][1])
            /// temp에는 (형태소[0] / 해당품사[1] 저장)
        //for 문을 돌면 all_temp에는 모든 문장의 형태소[0]/품사[1]가 저장 , embeddingmodel에는 모든 문장의 품사[1]가 저장
    all_temp.append(temp)
        //all_temp에는 temp저장
    embeddingmodel.append(temp_embedding)
        //embeddingmodel에는 temp_embedding 저장
    category_number_dic = {'IT과학': 0, '경제': 1, '정치': 2,'세계':3,'오피니언':4, '사회': 5, '생활문화': 6}
    all_temp.append(category_number_dic.get(category))
        ///all_temp에 카테고리 매핑정보까지 저장
    token.append(all_temp)
print("토큰 처리 완료")
    //여기까지 하면 all_temp에 형태소[0], 품사[1],매핑 정보 저장되어있음
 
embeddingmodel = []
for i in range(len(token)): ////index를 돌고
    temp_embeddingmodel = []
    for k in range(len(token[i][0])): ////날짜를 돌고
        temp_embeddingmodel.append(token[i][0][k])
    embeddingmodel.append(temp_embeddingmodel)
        ////돌면서 temp_embeddingmodel에 넣고
// max_vocab size 10000000 개당 1 GB 메모리 차지
embedding = Word2Vec(embeddingmodel, size=300, window=5, min_count=10, iter=5, sg=1, max_vocab_size = 360000000)
embedding.save('post.embedding') //만들어진 사각형 숫자 정보를 -> vector로 바꾸는 모델